In [9]:
import pandas as pd
import re
import json

In [10]:
# df2 = pd.read_excel(r'd:\Onedrive\OneDrive - UNI CONSULTING\Attachments\Code\uni_craw_tvpl\uni_craw_section_chapter_header\Craw_ko_en\ko_en_section_chapter_heading_des_notes.xlsx')

In [11]:
with open('output_file_2.txt', 'r', encoding='utf-8') as file:
    data = [json.loads(line.strip()) for line in file]
df = pd.DataFrame(data)

In [12]:
len(df)

1228

In [14]:
def create_section_df(df):
    # Select columns: 'ensection', 'ensection_note', 'kosection_note'
    df = df.loc[:, ['ensection', 'ensection_note', 'kosection_note']]
    # Drop duplicate rows across all columns
    df = df.drop_duplicates()
    return df

section_df = create_section_df(df.copy())

In [15]:
from bs4 import BeautifulSoup

def fix_atag(html):
    # Phân tích cú pháp HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Duyệt qua tất cả thẻ <a>
    for a_tag in soup.find_all('a'):
        # Kiểm tra nếu thẻ <a> chứa <img>
        if a_tag.find('img'):
            attributes_to_remove = ['rel', 'href', 'title','alt']
            for attr in attributes_to_remove:
                if attr in a_tag.attrs:  # Check if the attribute exists
                    del a_tag[attr]
            
        else:
            # Thay thẻ <a> bằng text bên trong nó
            a_tag.replace_with(a_tag.text)

    # Kết quả sau khi xử lý
    result = str(soup).replace('"/clip/','"https://unipass.customs.go.kr/clip/')
    # print(result)
    return result


In [16]:
def fig_a_clickable_tag(x):
    for i in range(len(x)):
        # Tạo đối tượng BeautifulSoup
        soup = BeautifulSoup(x[i], "html.parser")
        # Tìm tất cả các thẻ <a> có thuộc tính onClick
        a_tags = soup.find_all("a", attrs={"onclick": True})
        for a_tag in a_tags:
            # Thay thế thẻ <a> bằng nội dung văn bản của nó
            a_tag.replace_with(a_tag.get_text())
        # Cập nhật phần tử trong danh sách với nội dung đã thay thế
        x[i] = str(soup)
    return x

In [17]:
def process_section_notes(df,en_col,ko_col):
    
    def split_text(text):
        if not isinstance(text, str):
            return []
        text = text.replace('\n \n', '\n\n')
        text = text.replace('_x000D__x000D_', '\n')
        # Thay thế 3 hoặc nhiều '\n' thành '\n\n'
        text = re.sub(r'\n{2,}', '\n', text)
        delimiter = '\n'
        parts = text.split(delimiter)
        # print(parts)
        return parts
    
    def get_note_index(parts):
        note_pattern_en = r'^Notes?\.$'
        note_pattern_ko = r'^주:\s*'
        # Tìm các vị trí phần tử khớp với regex
        note_index = next((i for i, item in enumerate(parts) if re.fullmatch(note_pattern_en, item.replace('\n','').strip()) or re.fullmatch(note_pattern_ko, item.replace('\n','').strip())), None)
        # print("note_index: ",note_index)
        return note_index
    
    def get_general_index(parts):
        general_pattern_en = r'^GENERAL\.*$'
        general_pattern_ko = r'^총설\s*$'
        general_index = next((i for i, item in enumerate(parts) if re.fullmatch(general_pattern_en, item.replace('\n','').strip())or re.fullmatch(general_pattern_ko, item.replace('\n','').strip())), None)
        # print('general_index: ',general_index)
        return general_index
    
    def process_notes(x):
        split = split_text(x)
        note_idx = get_note_index(split)
        general_idx = get_general_index(split)
        if note_idx is None:
            return []
        elif general_idx is not None:
            return split[note_idx:general_idx]
        else:
            return split[note_idx:]
    
    def process_general(x):
        split = split_text(x)
        general_idx = get_general_index(split)
        if general_idx is None:
            return []
        elif general_idx is not None:
            return split[general_idx:]
        
    def process_des(x):
        split = split_text(x)
        note_idx = get_note_index(split)
        if note_idx is None:
            return split[1:]
        elif note_idx is not None:
            return split[1:note_idx]

    # Áp dụng hàm
    df['section_name'] = df[en_col].apply(lambda x: split_text(x)[0])
    df.insert(2, "hs", df["section_name"].str.split(" ").str[-1])
    df['en_description'] = df[en_col].apply(process_des)
    df['ko_description'] = df[ko_col].apply(process_des)
    
    df['en_notes'] = df[en_col].apply(process_notes)
    
    df['ko_notes'] = df[ko_col].apply(process_notes)
    
    df['en_general'] = df[en_col].apply(process_general)
    df['ko_general'] = df[ko_col].apply(process_general)


    return df


In [18]:
# Gọi hàm xử lý
section_df = process_section_notes(section_df, 'ensection_note', 'kosection_note')


In [19]:
section_df = section_df.drop(columns=['ensection_note', 'kosection_note'])

In [20]:
section_df['ko_notes'] = section_df['ko_notes'].apply(lambda x: [fix_atag(html) for html in x])
section_df['ko_general'] = section_df['ko_general'].apply(lambda x: [fix_atag(html) for html in x])
section_df['en_notes'] = section_df['en_notes'].apply(lambda x: [fix_atag(html) for html in x])
section_df['en_general'] = section_df['en_general'].apply(lambda x: [fix_atag(html) for html in x])

C:\Users\nguye\AppData\Local\Temp\ipykernel_14700\513519870.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')


In [21]:
section_df['ko_notes'] = section_df['ko_notes'].apply(lambda x: fig_a_clickable_tag(x))
section_df['ko_general'] = section_df['ko_general'].apply(lambda x: fig_a_clickable_tag(x))
section_df['ko_description'] = section_df['ko_description'].apply(lambda x: fig_a_clickable_tag(x))

C:\Users\nguye\AppData\Local\Temp\ipykernel_14700\1147944110.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(x[i], "html.parser")


In [24]:
section_df.head(2)

,ensection,hs,section_name,en_description,ko_description,en_notes,ko_notes,en_general,ko_general
0,01,I,Section I,"[LIVE ANIMALS; , ANIMAL PRODUCTS]",[살아 있는 동물과 동물성 생산품],"[Notes., 1.- Any reference in this Section to ...","[주: , 1. 이 부에 열거된 동물의 특정 속(屬)이나 종(種)에는 문맥상 달리 ...",[],[]
44,02,II,Section II,[VEGETABLE PRODUCTS ],[식물성 생산품],"[Note. , 1.- In this Section the term “pellets...","[주: , 1. 이 부에서 “펠릿(pellet)”이란 직접 압축하거나 전 중량의 1...",[],[]


In [23]:
section_df.to_excel("ko_en_section_notes_2024_2.xlsx",index = False)